In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import time
from ultralytics import YOLO
#from TCPdepth import closing
#from function_ObjDet  import getObject
from dimensionYolo import getObjectDimension 
from robotiq3f_py.robotiqcontrol.GripperController import GripperController
from yolo11Dimension import getObjectDimensiont
from closing import closing as closeGripper
#from robotiq3f_py.robotiqcontrol.GripperController import GripperController 
#from cam import cama
#from depthTracking import closing
#from depthTracking import gripTrigger

Funktion um Öffnungswinkel an Objektgröße anzupassen

In [ ]:
def openGripper(mode, x):

    if mode == 'Scissor':
        x = x + 1
        return max(0,min(190,int(-23.75 * x +237.5)))
    else:
        x = x+2
        print(x)
        return max(0,min(90,int(-7.7 * x +123))) # 90~3cm



Funktion um Werte in Register zu schreiben

In [ ]:
def gripperPos(pos):
    gripperPos = [pos]*3 
    return gripperPos


def gripperSpeed(speed):
    gripperSpeed = [speed]*3 
    return gripperSpeed

def gripperForce(force):
    gripperForce = [force]*3 
    return gripperForce


Initialize Gripper

In [ ]:
# Initialize the GripperController object with the IP address of the server
try:
    gripper = GripperController("192.168.1.11")
    gripper.activate()
    print('Connected')
except:
    print("No Connection to gripper")
    
'''
ACHTUNG: wird wohl immer verbunden
'''

Greifer in Ausgangslage bringen

In [ ]:
mode = 'Basic'

target_position = [0]*3
speed = [255] *3
force = [20] * 3
individual_control = False


gripper.command_gripper(rPRA=target_position, rSP=speed, rFR=force, rMOD=mode, rICF=individual_control)
print('command sent')

# for i in range(0,20):
#      time.sleep(0.25)
#      print([gripper.FingerA_Current, gripper.FingerB_Current, gripper.FingerC_Current]) 
#      print(f'position: {gripper.FingerA_Position}')
#      print(f'status_register: {gripper.gSTA}')
      


        # Wait for the gripper to reach the target position or object is detected
# while any([gripper.FingerA_Current, gripper.FingerB_Current, gripper.FingerC_Current]):
#         print('wait')
#         time.sleep(0.1)


Camera Set-Up

In [ ]:
# Auflösung definieren
x = 424
y = 240


dbr = 0.17873      # ausgerechnetes invlaid depth band für abstand von ca 15 cm
xdepth = int(dbr * x)
# Kamera Konfigurieren und starten
pipeline = rs.pipeline()
config = rs.config()
filter = rs.decimation_filter()
config.enable_stream(rs.stream.depth, x,y, rs.format.z16, 30)
config.enable_stream(rs.stream.color, x,y, rs.format.bgr8, 30)  # bgr is compatible with OpenCV 
pipelineProfile = pipeline.start(config)
# Align streams
align_to = rs.stream.color
align = rs.align(align_to)
# Set high accuracy mode
depth_sensor = pipelineProfile.get_device().first_depth_sensor()
depth_sensor.set_option(rs.option.visual_preset, rs.rs400_visual_preset.high_density)
#depth_sensor.set_option(rs.option.visual_preset, rs.rs400_visual_preset.medium_density)
depthScale = depth_sensor.get_depth_scale()

Warten bis Greifer Initialbewewegung ausgeführt hat

In [ ]:
try:
        while gripper.gSTA == 0:
                print('in Motion')
except Exception as e:
    print("Fehler aufgetreten:", e)
print('Gripper Ready')
#print(gripper.gSTA)

Object Detection

In [ ]:
model = YOLO("yolo11n.pt")
confidence_YOLO =0.35
label, xSize, ySize  = getObjectDimensiont(model,confidence_YOLO,pipeline, depthScale, align)

In [ ]:
# xSize = float(xSize)
# ySize = float(ySize)

In größte und kleinste Seite unterscheiden

In [ ]:
if xSize > ySize:
    maxSide = xSize
    minSide = ySize
else:
    maxSide = ySize
    minSide = xSize
maxSide, minSide

Entscheidungsbaum für Modus

In [ ]:
#objectsWithHole = ["toilet", "donut"]
# mit if Bedingung x == maxside methode fürs drehen
if maxSide >= 24:
    if minSide >= 15:
        print("Object too big!")
        mode = None
    else:
        mode = "Wide"
elif maxSide >= 12 :
    mode = "Basic"

elif   maxSide < 6 and (maxSide-minSide > 3 or minSide <3) :
    mode = "Scissor"
else:
    mode = "Pinch"
mode

Greifer Öfffnung an Objekt anpassen

In [ ]:

if mode:
        value = openGripper(mode,minSide)   
        force = gripperForce(20)
        speed = [255]
        target_position = gripperPos(value)
        
        individual_control = False
        #target_position = [250]*3
        #target_position, force, speed
else:
    individual_control = True
    mode = "Basic"
    target_position = [130,255,255]
    force = gripperForce(20)
    speed = [255]*3
    #target_position = gripperPos(value)
print(f'mode: {mode}')
print(f'position: {target_position}')
print(f'speed: {speed}')

Greifer in Greifposition fahren

In [ ]:
print(target_position, force, speed, individual_control,mode)

gripper.command_gripper(rPRA=target_position, rSP=speed, rFR=force, rMOD=mode, rICF=individual_control)
print('command sent')

time.sleep(1)
        # Wait for the gripper to reach the target position or object is detected
# while any([gripper.FingerA_Current, gripper.FingerB_Current, gripper.FingerC_Current]):
#         print('wait')
#         time.sleep(0.1)

while gripper.gSTA == 0:
        print('in Motion')

print('end')

Distanz des Objektes zu Greifer

In [ ]:
cv2.destroyAllWindows()

In [ ]:
if individual_control:
    print('No Mode')
else:
    grip = closeGripper(pipeline, depthScale, align, mode, x,y)
grip 

Commands to close gripper

In [ ]:
if grip:
        target_position = [255]*3 # Closing position
        if mode == "Basic" or mode == "Wide":
                speed = gripperSpeed(0)
        else:
                speed = gripperSpeed(10)
                
        
        gripper.command_gripper(rPRA=target_position, rSP=speed, rFR=force, rMOD=mode, rICF=individual_control)
        print('command sent')              
        # Wait for the gripper to reach the target position or object is detected
        # while any([gripper.FingerA_Current, gripper.FingerB_Current, gripper.FingerC_Current]):
        #         print('wait')
        #         time.sleep(0.1)
        while gripper.gSTA == 0:
                print('sleep')
        
        print('end')
else:
        print('nothing to grip')

In [ ]:
try:
        while gripper.gSTA == 0:
                print('in Motion')
except Exception as e:
    print("Fehler aufgetreten:", e)
print('Gripper Ready')
#print(gripper.gSTA)

Greifer nach x Sekunden öffnen

In [ ]:
time.sleep(5)
print('over')
mode = 'Basic'

target_position = [0]*3 # Opening position

speed = gripperSpeed(10)
        

gripper.command_gripper(rPRA=target_position, rSP=speed, rFR=force, rMOD=mode, rICF=individual_control)
print('command sent')              
# Wait for the gripper to reach the target position or object is detected
# while any([gripper.FingerA_Current, gripper.FingerB_Current, gripper.FingerC_Current]):
#         print('wait')
#         time.sleep(0.1)
while gripper.gSTA == 0:
        print('sleep')
gripper.command_gripper(rPRA=target_position, rSP=speed, rFR=force, rMOD=mode, rICF=individual_control)
print('end')

In [ ]:
pipeline.stop()

In [ ]:
gripper.close()
pipeline.stop()